# Extracting Resonance Parameter Covariances from an ENDF file

The covariance file has the same hierarchy as MF2: Section, Isotope, Energy range

## Reich-Moore with General Resolved Resonance Formats (LCOMP = 1)

In [ ]:
import ENDFtk

Pb208_tape = ENDFtk.tree.Tape.from_file( 'resources/n-Pb208.endf' )
Pb208_mat = Pb208_tape.materials.front()

Pb208_file32 = Pb208_mat.file(32).parse()
Pb208_section151 = Pb208_file32.section(151)
Pb208_resonance_region = Pb208_section151.isotopes[0]

Pb208_resonance_region

In [ ]:
print(f"ZA of this isotope (ZAI): {Pb208_resonance_region.ZAI}")
print(f"number of resonance ranges (NER): {Pb208_resonance_region.number_resonance_ranges}")

The RRR is the first range

In [ ]:
Pb208_RRR = Pb208_resonance_region.resonance_ranges[0]
Pb208_RRR

In [ ]:
print(f"Resonance range limits: {Pb208_RRR.lower_energy:.1E} - {Pb208_RRR.upper_energy:.1E} eV")
print(f"Type (LRU): {Pb208_RRR.type}")
print(f"Representation (LRF): {Pb208_RRR.representation}")
print(f"Energy-dependent scattering radius: {Pb208_RRR.energy_dependent_scattering_radius}")
print(f"Scattering radius calculation method (NAPS): {Pb208_RRR.scattering_radius_calculation_option}")

The parameters and covariance matrices are in the `parameters` attribute

In [ ]:
Pb208_parameters = Pb208_RRR.parameters
Pb208_parameters

In [ ]:
print(f"Target spin (SPI): {Pb208_parameters.spin}")
print(f"RMatrix formalism (LRF): {Pb208_parameters.representation}")
print(f"Covariance representation (LCOMP): {Pb208_parameters.covariance_representation}")
print(f"Scattering radius (AP): {Pb208_parameters.scattering_radius}")
print(f"There is scattering radius uncertainty (ISR): {Pb208_parameters.scattering_radius_uncertainty_flag}")
print(f"Number of short-range covariance blocks (NSRS): {Pb208_parameters.number_short_range_blocks}")
print(f"Number of long-range covariance blocks (NLRS): {Pb208_parameters.number_long_range_blocks}")

If there is scattering radius uncertainty, it is in the `scattering_radius_uncertainty` attribute

In [ ]:
Pb208_radius_unc = Pb208_parameters.scattering_radius_uncertainty
Pb208_radius_unc

In [ ]:
print(f"The default radius uncertainty (DAP): {Pb208_radius_unc.default_uncertainty}")
print(f"The l-dependent radius uncertainties (DAPL): {Pb208_radius_unc.uncertainties[:]}")

If there are short-range covariance blocks, the format is dependent on the RMatrix formalism used. For Reich-Moore, the parameters from File 2 are repeated along with the covariance matrix as an upper-triangular matrix.

In [ ]:
Pb208_short_range = Pb208_parameters.short_range_blocks[0]
Pb208_short_range

In [ ]:
print(f"Number of resonances (NRB): {Pb208_short_range.number_resonances}")
print(f"For each resonances, number of parameters with covariances (MPAR): {Pb208_short_range.number_parameters_with_covariances}")
print(f"Covariance matrix order (NPARB): {Pb208_short_range.covariance_matrix_order}   (which equals NRB*MPAR)")
print(f"Number of values in the upper-triangular matrix (NVS): {Pb208_short_range.number_values}   (which equals NPARB * (NPARB + 1) / 2)")

The parameters with uncertainties (for all spin groups) are included:

In [ ]:
print(f"Energy [eV] \t J \t Gamma_n \t Gamma_g \t Gamma_f1 \t Gamma_f2")
print("-"*82)
for Er, J, n, g, f1, f2 in zip(Pb208_short_range.resonance_energies,
                               Pb208_short_range.spin_values, 
                               Pb208_short_range.neutron_widths, 
                               Pb208_short_range.gamma_widths, 
                               Pb208_short_range.first_fission_widths, 
                               Pb208_short_range.second_fission_widths):
    print(f"{Er:.2E} \t {J} \t {n}    \t {g}    \t {f1}   \t         {f2}")

And the covariance matrix is presented as an upper-triangular matrix in list form

In [ ]:
Pb208_short_range.covariance_matrix[:10]

The list can be converted to a full symmetric matrix using numpy functions:

In [ ]:
import numpy as np

def fill_in_matrix(upper_triangular_list, matrix_order):

    # check that the inputs are consistent
    if not len(upper_triangular_list) == matrix_order * (matrix_order + 1) / 2:
        raise ValueError(f"The length of the matrix elements is not consistent with the stated matrix order")

    # create empty matrix
    covariance_matrix = np.zeros((matrix_order, matrix_order))

    # get the indices of the upper triangular values
    indices = np.triu_indices(matrix_order)

    # fill in the matrix
    covariance_matrix[indices] = upper_triangular_list

    # fill in the lower triangle
    covariance_matrix += np.triu(covariance_matrix,k=1).T

    return covariance_matrix

cov = fill_in_matrix(Pb208_short_range.covariance_matrix, Pb208_short_range.covariance_matrix_order)

print(cov[:5,:5])

The order of the covariance matrix is the set of resonance parameters for each resonance:

$$
ER_1, GN_1, GG_1, GFA_1, GFB_1, ER_2, GN_2, GG_2, GFA_2, GFB_2, ..., ER_{NRB}, GN_{NRB}, GG_{NRB}, GFA_{NRB}, GFB_{NRB}
$$

where $NRB$ is the number of resonances with uncertainty. In this case the fission widths are not included, which is indicated by the fact that the flag MPAR (for each resonances, number of parameters with covariances) is 3


This list can be created in Python with zip and sum:

In [ ]:
# this creates a tuple
parameter_list = sum(
    zip(
        Pb208_short_range.resonance_energies,
        Pb208_short_range.neutron_widths,
        Pb208_short_range.gamma_widths,
        # Pb208_short_range.first_fission_widths,   # not included here because MPAR = 3
        # Pb208_short_range.second_fission_widths   # not included here because MPAR = 3
        ),
        ()
)

parameter_list = np.array(parameter_list)
parameter_list

from this the relative uncertainties can be determined:

In [ ]:
relative_uncertainties = np.sqrt(np.diag(cov)) / parameter_list

In [ ]:
import matplotlib.pyplot as plt

_  = plt.hist(100*relative_uncertainties,bins = 100)
plt.ylabel("Frequency")
plt.yscale("log")
plt.xlabel("Relative Uncertainty [%]")

## R-Matrix Limited with Resolved Resonance Compact Covariance Format  (LCOMP = 2)

In [ ]:

Rh103_tape = ENDFtk.tree.Tape.from_file( 'resources/n-Rh103.endf' )
Rh103_mat = Rh103_tape.materials.front()

Rh103_file32 = Rh103_mat.file(32).parse()
Rh103_section151 = Rh103_file32.section(151)
Rh103_resonance_region = Rh103_section151.isotopes[0]

Rh103_resonance_region

In [ ]:
print(f"ZA of this isotope (ZAI): {Rh103_resonance_region.ZAI}")
print(f"number of resonance ranges (NER): {Rh103_resonance_region.number_resonance_ranges}")

The RRR is the first region

In [ ]:
Rh103_RRR = Rh103_resonance_region.resonance_ranges[0]
Rh103_RRR

In [ ]:
print(f"Resonance range limits: {Rh103_RRR.lower_energy:.1E} - {Rh103_RRR.upper_energy:.1E} eV")
print(f"Type (LRU): {Rh103_RRR.type}")
print(f"Representation (LRF): {Rh103_RRR.representation}")
print(f"Energy-dependent scattering radius: {Rh103_RRR.energy_dependent_scattering_radius}")
print(f"Scattering radius calculation method (NAPS): {Rh103_RRR.scattering_radius_calculation_option}")

The parameters and covariance matrices are in the `parameters` attribute

In [ ]:
Rh103_parameters = Rh103_RRR.parameters
Rh103_parameters

In [ ]:
print(f"RMatrix formalism (LRF): {Rh103_parameters.representation}")
print(f"Covariance representation (LCOMP): {Rh103_parameters.covariance_representation}")
print(f"There is scattering radius uncertainty (ISR): {Rh103_parameters.scattering_radius_uncertainty_flag}")
print(f"The scattering radius uncertainty (DAP): { Rh103_parameters.scattering_radius_uncertainty }")
print(f"The widths are in units [eV^1/2] instead of [eV] (IFG): {Rh103_parameters.reduced_widths}")
print(f"The number of spin groups (NJS): {Rh103_parameters.number_spin_groups}")

The parameters and uncertainties are in the `uncertainties` attribute

In [ ]:
Rh103_uncertainties = Rh103_parameters.uncertainties
Rh103_uncertainties

In [ ]:
print(f"Number of spin groups (NJSX): {Rh103_uncertainties.number_spin_groups}")

The particle pair information is repeated in `particle_pairs`, and the parameters are in the `spin_groups` attribute

In [ ]:
Rh103_swave = Rh103_uncertainties.spin_groups[0]
Rh103_swave

In [ ]:
print(f"Number of channels (NCH): {Rh103_swave.number_channels}")
print(f"Number of resonances (NRSA): {Rh103_swave.number_resonances}")
print(f"Spin of this group (AJ): {Rh103_swave.spin}")
print(f"Parity of this group (PJ): {Rh103_swave.parity}")

In [ ]:
Rh103_swave_parameters = Rh103_swave.parameters
Rh103_swave_parameters

In [ ]:
print(f"Number of resonances (NRSA): {Rh103_swave_parameters.number_resonances}")

The energies and their uncertainties are in `resonance_energies` and `resonance_energy_uncertainties`:

In [ ]:
for En, dEn in zip(Rh103_swave_parameters.resonance_energies, Rh103_swave_parameters.resonance_energy_uncertainties):
    print(f'{En} +/-  {dEn} eV')

The resonance parameters are in the analogous attributes:

In [ ]:
Rh103_swave_parameters.resonance_parameters[0][:], Rh103_swave_parameters.resonance_parameter_uncertainties[0][:]

The lists of values and uncertainties can be assembled with numpy arrays:

In [ ]:
parameter_values = np.array(Rh103_swave_parameters.resonance_parameters).T
parameter_values = np.vstack([Rh103_swave_parameters.resonance_energies,parameter_values]).T
parameter_values = parameter_values.reshape((parameter_values.shape[0]*parameter_values.shape[1]))
parameter_values[:10]

In [ ]:
parameter_uncertainties = np.array(Rh103_swave_parameters.resonance_parameter_uncertainties).T
parameter_uncertainties = np.vstack([Rh103_swave_parameters.resonance_energy_uncertainties,parameter_uncertainties]).T
parameter_uncertainties = parameter_uncertainties.reshape((parameter_uncertainties.shape[0]*parameter_uncertainties.shape[1]))
parameter_uncertainties[:10]

from this, the relative uncertainties can be obtained:

In [ ]:
Rh103_relative_uncertainties = parameter_uncertainties / np.abs(parameter_values)
Rh103_relative_uncertainties[:10]

In this format, the correlation matrix is stored rather than the covariance matrix

In [ ]:
Rh103_correlation_object = Rh103_parameters.correlation_matrix
Rh103_correlation_object

In [ ]:
print(f"The order of the correlation matrix (NNN): {Rh103_correlation_object.order}")
print(f"The number of digits used for each correlation value (NDIGIT): {Rh103_correlation_object.number_digits}")

The indices are stored in the attributes `I` and `J`, and the correlation values are in `correlations`

In [ ]:
row_indices = np.array(Rh103_correlation_object.I) - 1  # Values are 1-indexed
col_indices = np.array(Rh103_correlation_object.J) - 1  # Values are 1-indexed
values = np.array(Rh103_correlation_object.correlations)

row_indices[0], col_indices[0], values[0]

the correlation matrix can be constructed from these indices and values:

In [ ]:
Rh103_correlation_matrix = np.zeros((Rh103_correlation_object.order,Rh103_correlation_object.order))

Rh103_correlation_matrix[row_indices,col_indices] = values

# fill in the upper triangle
Rh103_correlation_matrix += np.tril(Rh103_correlation_matrix,k=1).T

# add ones along the diagonal
Rh103_correlation_matrix += np.identity(Rh103_correlation_object.order)

Rh103_correlation_matrix[:6,:6]

In [ ]:
# this takes a little while to produce

plt.pcolor(Rh103_correlation_matrix)
plt.xlabel("Parameter index")
plt.ylabel("Parameter index")
plt.colorbar()

The covariance matrix can be constructed from the uncertainties and the correlation matrix. The parameters for each spin group need to be collected into the parameter and uncertainty arrays:

In [ ]:
Rh103_parameter_values = np.array([])
Rh103_parameter_uncertainties = np.array([])

for spin_group in Rh103_uncertainties.spin_groups:

    parameter_values = np.array(spin_group.parameters.resonance_parameters).T
    parameter_values = parameter_values[:spin_group.NCH,:]  # grab only the number of channels indicated by NCH
    parameter_values = np.vstack([spin_group.parameters.resonance_energies,parameter_values]).T
    parameter_values = parameter_values.reshape((parameter_values.shape[0]*parameter_values.shape[1]))

    Rh103_parameter_values = np.append(Rh103_parameter_values, parameter_values)

    parameter_uncertainties = np.array(spin_group.parameters.resonance_parameter_uncertainties).T
    parameter_uncertainties = parameter_uncertainties[:spin_group.NCH,:]  # grab only the number of channels indicated by NCH
    parameter_uncertainties = np.vstack([spin_group.parameters.resonance_energy_uncertainties,parameter_uncertainties]).T
    parameter_uncertainties = parameter_uncertainties.reshape((parameter_uncertainties.shape[0]*parameter_uncertainties.shape[1]))

    Rh103_parameter_uncertainties = np.append(Rh103_parameter_uncertainties, parameter_uncertainties)



Rh103_parameter_values.shape, Rh103_parameter_uncertainties.shape

In [ ]:
Rh103_covariance_matrix = np.diag(Rh103_parameter_uncertainties) @ Rh103_correlation_matrix @ np.diag(Rh103_parameter_uncertainties)

Rh103_covariance_matrix[3:6,3:6]

check the first non-zero correlation value:

In [ ]:
Rh103_covariance_matrix[4,1]

In [ ]:
Rh103_parameter_uncertainties[4] * Rh103_correlation_matrix[4,1] *  Rh103_parameter_uncertainties[1]

In [ ]:
np.isclose(Rh103_covariance_matrix[4,1], Rh103_parameter_uncertainties[4] * Rh103_correlation_matrix[4,1] *  Rh103_parameter_uncertainties[1])